# Load Codebase Indices for Evaluation Run

First, you will need the chroma codebase indices for the evaluation run. You can download the indices from GCS via:

```sh
mkdir -p ./data/chroma && gsutil cp -r gs://sentry-ml/tmp_jenn/chroma/chroma ./data
```

Second, get the postgres dumps:
```sh
gsutil cp -r gs://sentry-ml/tmp_jenn/chroma/namespaces.json ./data/chroma
gsutil cp -r gs://sentry-ml/tmp_jenn/chroma/repo_infos.json ./data/chroma
```

Then, you can load the repositories and namespace rows into postgres with the following cell:

In [ ]:
import json
from seer.automation.codebase.models import RepositoryInfo, CodebaseNamespace
from seer.db import Session

def load_json_into_db(repo_info_file='../data/repo_infos.json', namespace_file='../data/namespaces.json'):
    with open(repo_info_file, 'r') as f:
        repo_infos_json = json.load(f)
    
    with open(namespace_file, 'r') as f:
        namespaces_json = json.load(f)
    
    with Session() as session:
        for repo_info in repo_infos_json:
            db_repo_info = RepositoryInfo.model_validate_json(repo_info).to_db_model()
            session.merge(db_repo_info)

        session.flush()
        
        for namespace in namespaces_json:
            db_namespace = CodebaseNamespace.model_validate_json(namespace).to_db_model()
            session.merge(db_namespace)
        
        session.commit()

load_json_into_db()
